# Movielens DB 만들기

In [1]:
import mysql.connector
import pandas as pd
from tqdm import tqdm

In [2]:
# db connector 정의
db_conn = mysql.connector.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    passwd = '0000'
)
cursor = db_conn.cursor()

In [3]:
# DB 생성
cursor.execute('CREATE DATABASE ml100k;')
# rating table 생성
cursor.execute('CREATE TABLE ml100k.rating(\
            	user_id INT,\
                movie_id INT,\
                rating INT,\
                ux_tsmp INT\
            )')
db_conn.commit()

# file open
sql = 'INSERT INTO ml100k.rating (user_id, movie_id, rating, ux_tsmp) VALUES (%s, %s, %s, %s);' 
line = []
with open('rating.csv', 'rt') as f:
    for l in f:
        line.append(l.rstrip().split())
    cursor.executemany(sql, line)
    db_conn.commit()
    
cursor.execute("SELECT * FROM ml100k.rating")
myresult = cursor.fetchall()
myresult

In [ ]:
# user table 생성
cursor.execute('CREATE TABLE ml100k.user(\
	user_id INT PRIMARY KEY,\
    age INT,\
    gender CHAR,\
    occupation VARCHAR(20),\
    zip_code VARCHAR(10)\
    );')
db_conn.commit()

# file open
sql = 'INSERT INTO ml100k.user (user_id, age, gender, occupation, zip_code) VALUES (%s, %s, %s, %s, %s);' 
line = []
with open('user.csv', 'rt') as f:
    for l in f:
        line.append(l.rstrip().split('|'))
    cursor.executemany(sql, line)
    db_conn.commit()
    
cursor.execute("SELECT * FROM ml100k.user")
myresult = cursor.fetchall()
myresult

In [ ]:
# movie table 생성
cursor.execute('CREATE TABLE ml100k.movie(\
	movie_id INT PRIMARY KEY,\
    movie_title VARCHAR(255),\
    release_date VARCHAR(20),\
    video_release_date VARCHAR(20),\
    IMDb_URL VARCHAR(255),\
    unknown BOOL,\
    Action BOOL,\
    Adventure BOOL,\
    Animation BOOL,\
    Children BOOL,\
    Comedy BOOL,\
    Crime BOOL,\
    Documentary BOOL,\
    Drama BOOL,\
    Fantasy BOOL,\
    FilmNoir BOOL,\
    Horror BOOL,\
    Musical BOOL,\
    Mystery BOOL,\
    Romance BOOL,\
    SciFi BOOL,\
    Thriller BOOL,\
    War BOOL,\
    Western  BOOL\
);')
db_conn.commit()

# file open
sql = 'INSERT INTO ml100k.movie VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);' 
line = []
with open('movie2.csv', 'rt') as f:
    ii = 0
    for l in f:
        line.append(l.rstrip().split('|')) 
        ii += 1
        if(ii % 5000 == 0):
            line = []
            cursor.executemany(sql, line)
            db_conn.commit()
cursor.executemany(sql, line)
db_conn.commit()

cursor.execute("SELECT * FROM ml100k.movie")
myresult = cursor.fetchall()
myresult

In [35]:
# 외래키 설정
cursor.execute('ALTER TABLE ml100k.rating ADD FOREIGN KEY(user_id) REFERENCES ml100k.user(user_id) ON DELETE CASCADE;')
cursor.execute('ALTER TABLE ml100k.rating ADD FOREIGN KEY(movie_id) REFERENCES ml100k.movie(movie_id) ON DELETE CASCADE;')
db_conn.commit()

In [38]:
movie_df = pd.read_csv('movie.csv', sep = '|', encoding='latin1', header = None)
movie_df.columns = ['movie_id' , 'movie_title' , 'release_date' , 'video_release_date' ,
              'IMDb_URL' , 'unknown' , 'Action' , 'Adventure' , 'Animation' ,
              'Children' , 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western']

In [39]:
movie_df

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
monthMap = {'Jan' : 1, 'Feb' : 2, 'Mar' : 3, 'Apr' : 4, 'May' : 5, 'Jun' : 6, 'Jul' : 7, 'Aug' : 8, 'Sep' : 9, 'Oct' : 10, 'Nov' : 11, 'Dec' : 12}
def getYear(data):
    s = data.split('-')
    if len(s) > 1:
        return s[2]
    else:
        print
movie_df['rlsYear'] = movie_df['release_date'].apply(lambda x : str(x).split('-')[2])

In [65]:

movie_df['rlsYear'] = movie_df['release_date'].apply(lambda x : str(x).split('-')[2])
movie_df['rlsMonth'] = movie_df['release_date'].apply(lambda x : str(x).split('-')[1])
movie_df['rlsMonth'] = movie_df['rlsMonth'].map(monthMap)
movie_df['rlsDay'] = movie_df['release_date'].apply(lambda x : str(x).split('-')[0])

,movie_id,movie_title,release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,Comedy,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rlsYear,rlsMonth,rlsDay
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,1995,1,01
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,1,0,0,1995,1,01
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1995,1,01
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,1995,1,01
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,1,0,0,1995,1,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1998,2,06
1678,1679,B. Monkey (1998),06-Feb-1998,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1998,2,06
1679,1680,Sliding Doors (1998),01-Jan-1998,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,...,0,0,1,0,0,0,0,1998,1,01
1680,1681,You So Crazy (1994),01-Jan-1994,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1994,1,01


In [68]:
movie_df.to_csv('movie2.csv', index = False)

In [73]:
cursor.execute('ALTER TABLE ml100k.rating DROP FOREIGN KEY rating_ibfk_2;')
cursor.execute('DROP TABLE ml100k.movie;')
db_conn.commit()